In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
 %%javascript
IPython.OutputArea.auto_scroll_threshold = 10

<IPython.core.display.Javascript object>

# Business questions
In iAuditor, our customers go through __creating/applying__ public *templates* (checklist), __using__ the *template* for their inspections, __creating actions__ from issues found through the inspections and __reporting incidents/accidents__ whenever necessary. Our customers come from a __variety of industries__ with a large proportion contributed by food & hospitality, construction, and manufacturing. With the provided data,
1. We’re interested in seeking solutions that help increase the number of customers using the standard public checklist templates for their inspections (e.g. how can we recommend the suitable templates to customers)?
2. Based on the findings above, do you have any suggestions on potential features/products that can be built to improve our customers’ experiences ?
3. Through using the checklists for inspection, can you suggest some potential solutions for correlating the data from the checklists with potential risks relevant to the respective inspections.



To address these questions I did an analysis of the dataset from the point of view of quantity and quality.

# Data Analysis

In [3]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from collections import defaultdict
from collections import Counter
from pandas.io.json import json_normalize

plt.style.use('seaborn-whitegrid')
INPUT_FILE = "/Users/silvia/Downloads/sample_pl_data.json"

I load the data in two formats as dataframe and as json. It's not optimal but it was the easiest approach as the data is small.

### Load Data JSON
The data in Json format allows me to handle the data easily.

In [4]:
with open(INPUT_FILE, "r") as fin:
    data_json = json.load(fin)
    
print(f"Data loaded: {len(data_json)} items")

Data loaded: 170 items


### Load Data Dataframe
The data in dataframe format allows me to have a look over the information.

In [5]:
with open(INPUT_FILE, "r") as fin:
    data_df = pd.read_json(fin)
data_df.shape

(170, 24)

In [6]:
data_df.head()

,revision_key,_rev,created_at,type,export_profiles,template_data,deleted,permissions,action_item_profiles,autoshares,...,items,trashed,temp_rev,meta,assets,migrated_at,_id,revision_id,libraryId,server_revision_key
0,36b7d0d5-d599-437f-97ca-27d31db910b6,6-96455a285bbe4176b2d3e5433645de76,2018-04-27 01:18:26.079000+00:00,template,{},{'metadata': {'audit_title_rule': ['f3245d40-e...,False,{'owner': 'user_80a0569c75c211e49ed3001b1118ce...,{},{'user_de146dd7a04011e4b27f001b1118ce11': {'vi...,...,[{'item_id': '95594086-34f0-4680-b43b-78fa330c...,False,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,efcef146-0022-4966-b15c-9df6e79f88ea,NaN,2018-06-29 05:55:32.658000+00:00,template,{},{'metadata': {'audit_title_rule': ['f3245d40-e...,False,{'owner': 'user_de146dd7a04011e4b27f001b1118ce...,{},NaN,...,[{'item_id': 'e6384211-e707-484a-96fb-98664b17...,False,1-c8c995170f7b422a90c3c9ac48605f08,{'rev': '1821-153c8bdc61f000000000000000000000...,NaN,NaT,NaN,NaN,NaN,NaN
2,1c6267d8-8f3d-46f5-a021-81a4e77630dd,NaN,2018-08-02 04:14:53.311000+00:00,template,NaN,{'metadata': {'audit_title_rule': ['f3245d40-e...,False,{'owner': 'user_7d7e66c7db3e4c8387e31cbcc81323...,{},NaN,...,[{'item_id': '6e1cb72b-389e-40e3-9779-f47e8574...,False,1-148ecc06624b4ee8ad11bb9f3ab29aef,{'rev': '16925-1546f61cdb540000000000000000000...,[],NaT,NaN,NaN,NaN,NaN
3,f23143dd-d0f5-40d6-90ee-c4dcb91fcd51,NaN,2018-04-18 01:14:14.893000+00:00,template,None,{'metadata': {'audit_title_rule': ['f3245d42-e...,False,{'owner': 'user_1d8fa6fdca154b42a0d8b1bcf9b720...,NaN,NaN,...,[{'item_id': '9eb6457e-8f93-4158-808a-11e36612...,True,NaN,{'rev': '1721-15266f12dd4100000000000000000000...,NaN,NaT,NaN,NaN,NaN,NaN
4,63948F9D-9710-4DF0-888A-4C046BEED772,NaN,2018-07-23 04:28:54.832000+00:00,template,NaN,"{'metadata': {'image': '', 'doc_no': '[number]...",False,{'owner': 'user_935cec3a55a211e39f35001b1118ce...,{},NaN,...,[{'item_id': '190E2CBC-B49B-4961-B13A-DF4162E4...,False,1-56631843d31848e49ba04f8f2ec61a7f,"{'rev': '52-1543e50b8c4300000000000000000000',...",NaN,NaT,NaN,NaN,NaN,NaN


I want to see the attributes of the templates and which are the most common.

In [7]:
def get_counts(gen):
    counts = Counter(gen)

    df_counts = pd.DataFrame(counts.items(), columns=["Keys", "Freq."])
    df_counts.set_index("Keys", inplace=True)
    df_counts.sort_values(by="Freq.", ascending=False, inplace=True)
    return df_counts

In [8]:
df_counts = get_counts((key for template in data_json for key in template))
df_counts

,Freq.
Keys,
revision_key,170
permissions,170
trashed,170
items,170
modified_at,170
header,170
template_id,170
deleted,170
template_data,170


I suppose that attributes that occur in every template are more important. Between them, I think the most important for our task are: 

* template_id
* items
* header
* template_data
* name

In [9]:
data_df[["template_id","items", "header", "template_data", "name"]]

,template_id,items,header,template_data,name
0,template_65db0caaac874d08ae7e7d15d05b5c7c,[{'item_id': '95594086-34f0-4680-b43b-78fa330c...,[{'item_id': '6f6f5e7f-9787-4a06-a4bc-7baf1c5f...,{'metadata': {'audit_title_rule': ['f3245d40-e...,All items - duplicate
1,template_dcb88e21ea304c9baa842daa2e5abafc,[{'item_id': 'e6384211-e707-484a-96fb-98664b17...,[{'item_id': '2bc84d28-3945-4110-9a35-7f8e9f88...,{'metadata': {'audit_title_rule': ['f3245d40-e...,Restest - duplicate
2,template_58aa02d963444fd9b94c76ea03537d9f,[{'item_id': '6e1cb72b-389e-40e3-9779-f47e8574...,[{'item_id': 'ac4b784a-69d1-43d8-9dbb-a85a41d8...,{'metadata': {'audit_title_rule': ['f3245d40-e...,bda04557-fed0-4f60-ad3e-ad1d590b79b1
3,template_c0ed72130e4f401eb281c5d7e725ec64,[{'item_id': '9eb6457e-8f93-4158-808a-11e36612...,[{'item_id': '074d7af2-b04d-4247-8a00-d5aff6e6...,{'metadata': {'audit_title_rule': ['f3245d42-e...,Question and List
4,template_0D81EB72BFBD4D39ABC14BBB7735691F,[{'item_id': '190E2CBC-B49B-4961-B13A-DF4162E4...,[{'item_id': 'D3CBEBF5-AC48-4B3B-812D-6E7A302A...,"{'metadata': {'image': '', 'doc_no': '[number]...",Advanced Items Template
...,...,...,...,...,...
165,template_CB3D831B3C25481896BEC6B2C0ED8D57,[{'item_id': '5e93bab0-d8e0-11e2-9b70-a5eba55b...,[{'item_id': 'D16BE174-8E40-4839-9C22-288C5AEF...,{'metadata': {'audit_title_rule': ['f3245d40-e...,Rapport van expertise - WB
166,template_4fcce8994e9f453c8f87ed7eccc41591,[{'item_id': 'e6352487-f07e-4af9-8390-cf59d29b...,[{'item_id': '5c76b3dd-f3f8-4988-9e43-cca7887a...,{'metadata': {'audit_title_rule': ['f3245d42-e...,Grs test 2
167,template_c814e26f64944ee1bde8917a1f3587e2,[{'item_id': 'b07a6cd0-29b9-11e5-892f-a3a8eaef...,[{'item_id': 'f3245d39-ea77-11e1-aff1-0800200c...,{'metadata': {'audit_title_rule': ['f3245d40-e...,The new DR!
168,template_6E2570F305DB4109A32863669C0018D0,[{'item_id': '31CCC382-7242-4C27-8ACD-4E5F2323...,[{'item_id': '474038C6-F8F9-4880-BFA5-A5E9D68F...,{'metadata': {'audit_title_rule': ['f3245d40-e...,FFVA Loss Control Recommendation Letter [EM] -...


*Template_data*, *Header* and *Items* need further investigation.

#### Template data

In [10]:
counts_template_data = get_counts((key for template in data_json for key in template["template_data"]))
counts_template_data

,Freq.
Keys,
metadata,170
versioning,170
authorship,170
media,170
metrics,170
response_sets,170
author,170
mandatory_mark_as_complete,122
is_scoring_enabled,64


In [11]:
counts_metadata = get_counts((key for template in data_json for key in template["template_data"]["metadata"]))
counts_metadata

,Freq.
Keys,
image,170
name,170
description,170
industry,170
subindustry,170
doc_no_suffix,162
doc_no_prefix,162
doc_no,157
audit_title_rule,130


In [12]:
df_metadata = pd.concat([json_normalize(template["template_data"]["metadata"]) for template in data_json], ignore_index = bool)
df_metadata

/Users/silvia/PycharmProjects/SafetyCulture/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,audit_title_rule,description,doc_no,doc_no_prefix,doc_no_suffix,image,industry,name,subindustry,tags
0,[f3245d40-ea77-11e1-aff1-0800200c9a66],,[number],NaN,NaN,,-2,All items - duplicate,-2,NaN
1,[f3245d40-ea77-11e1-aff1-0800200c9a66],,[number],,,70b4ee9d-1a8f-4eda-bd74-d2704011d018,-2,Restest - duplicate,-2,NaN
2,[f3245d40-ea77-11e1-aff1-0800200c9a66],,[number],,,,-2,bda04557-fed0-4f60-ad3e-ad1d590b79b1,-2,NaN
3,"[f3245d42-ea77-11e1-aff1-0800200c9a66, f3245d4...",,[number],NaN,NaN,,-2,Question and List - duplicate,-2,NaN
4,NaN,Beta Testing Template,[number],,,,-2,Advanced Items Template,-2,NaN
...,...,...,...,...,...,...,...,...,...,...
165,[f3245d40-ea77-11e1-aff1-0800200c9a66],Versie 24 juni 2015,[number],,,,7,Rapport van expertise - WB,3,NaN
166,"[f3245d42-ea77-11e1-aff1-0800200c9a66, f3245d4...",,[number],,,,7,Grs test 2,3,NaN
167,[f3245d40-ea77-11e1-aff1-0800200c9a66],ewfwef,[number],,,,7,The new DR!,3,NaN
168,[f3245d40-ea77-11e1-aff1-0800200c9a66],"1.22.15 - Recommendation Letter, with typing b...",,,,1DD4B731-0D1C-43FE-A5D3-71739B728D45,7,FFVA Loss Control Recommendation Letter [EM] -...,3,NaN


Whats the difference between *template\[name\]* and *template\[template_data\]\[metadata\]\[name\]*. 

In [13]:
metadata_name = data_df["template_data"].apply(lambda x: x['metadata']['name']).rename("template_name")
metadata_description = data_df["template_data"].apply(lambda x: x['metadata']['description']).rename("template_description")
df_names = pd.concat([data_df["name"], metadata_name, metadata_description], axis=1)
df_names[df_names["name"] != df_names["template_name"]]

,name,template_name,template_description
3,Question and List,Question and List - duplicate,
16,Question and List,Question and List - duplicate,
21,NaN,,
22,NaN,,
23,NaN,,
24,NaN,,
26,NaN,,
27,NaN,,
28,NaN,,
32,NaN,,


These two columns contain duplicate information, but for 17 templates. 14 out of these 17 templates are empty but the *name* column is filled with NaN while *template_name* is filled with an empty string. I decide to ignore *name* column.

In [14]:
counts_metrics = get_counts((key for template in data_json for key in template["template_data"]["metrics"]))
counts_metrics

,Freq.
Keys,
avg_duration,170
use_count,170
date_last_used,170
rating,170
est_duration,120
duration_count,84


In [15]:
df_metrics = pd.concat([json_normalize(template["template_data"]["metrics"]) for template in data_json], ignore_index = bool)
df_metrics

/Users/silvia/PycharmProjects/SafetyCulture/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,avg_duration,date_last_used,duration_count,est_duration,rating,use_count
0,0,1.555284e+09,NaN,0.0,-1,12
1,0,1.505284e+09,NaN,0.0,-1,4
2,-1,1.475626e+09,NaN,-1.0,-1,1
3,0,-1.000000e+00,NaN,0.0,-1,0
4,-1,-1.000000e+00,NaN,-1.0,-1,0
...,...,...,...,...,...,...
165,-1,-1.000000e+00,NaN,-1.0,-1,0
166,0,1.557378e+09,0.0,NaN,-1,3
167,0,0.000000e+00,NaN,-1.0,0,0
168,-1,-1.000000e+00,NaN,-1.0,-1,0


In [16]:
df_metrics.describe()

,avg_duration,date_last_used,duration_count,est_duration,rating,use_count
count,170.000000,1.700000e+02,84.0,120.000000,170.000000,170.000000
mean,-0.523529,2.911996e+08,0.0,-0.866667,-0.670588,0.394118
std,0.500922,5.851567e+08,0.0,0.341360,0.471388,1.755118
min,-1.000000,-1.000000e+00,0.0,-1.000000,-1.000000,0.000000
25%,-1.000000,-1.000000e+00,0.0,-1.000000,-1.000000,0.000000
50%,-1.000000,0.000000e+00,0.0,-1.000000,-1.000000,0.000000
75%,0.000000,0.000000e+00,0.0,-1.000000,0.000000,0.000000
max,0.000000,1.557721e+09,0.0,0.000000,0.000000,13.000000


We can see that *use_count* is the only metric that is filled in for a few templates. 

In [17]:
df_metrics["use_count"][df_metrics["use_count"]>0].shape

(22,)

Only 22 templates have use_count largen than 0. 

#### Header

In [18]:
from pandas.io.json import json_normalize
df_header = pd.concat([json_normalize(template["header"]) for template in data_json], ignore_index = bool)
df_header

/Users/silvia/PycharmProjects/SafetyCulture/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,action_item_profile_id,inactive,item_id,label,options.condition,options.drawing_base_image,options.element,options.enable_date,options.enable_signature_timestamp,options.enable_time,...,options.type,options.values,options.visible_in_audit,options.visible_in_report,options.weighting,parent_id,reference_item_profile_ids,responses.datetime,responses.value,type
0,NaN,NaN,6f6f5e7f-9787-4a06-a4bc-7baf1c5f23b3,Title Page,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,None,NaN,NaN,NaN,section
1,NaN,NaN,f3245d46-ea77-11e1-aff1-0800200c9a66,Document No.,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,6f6f5e7f-9787-4a06-a4bc-7baf1c5f23b3,NaN,NaN,NaN,textsingle
2,NaN,NaN,f3245d40-ea77-11e1-aff1-0800200c9a66,Audit Title,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,6f6f5e7f-9787-4a06-a4bc-7baf1c5f23b3,NaN,NaN,NaN,textsingle
3,NaN,NaN,f3245d41-ea77-11e1-aff1-0800200c9a66,Client / Site,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,6f6f5e7f-9787-4a06-a4bc-7baf1c5f23b3,NaN,NaN,NaN,textsingle
4,NaN,NaN,f3245d42-ea77-11e1-aff1-0800200c9a66,Conducted on,NaN,NaN,NaN,True,NaN,True,...,NaN,NaN,NaN,NaN,1.0,6f6f5e7f-9787-4a06-a4bc-7baf1c5f23b3,NaN,NaN,NaN,datetime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,NaN,NaN,f3245d41-ea77-11e1-aff1-0800200c9a66,Store Name:,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6703ACD4-2DFC-4F23-AFBC-5084D65722E2,NaN,NaN,NaN,text
1185,NaN,NaN,27C5FD11-DD48-4A15-A5AE-C3222C60CFF5,Project Manager:,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6703ACD4-2DFC-4F23-AFBC-5084D65722E2,NaN,NaN,NaN,text
1186,NaN,NaN,A8F9CC14-4BCC-414E-8EA4-12F94EBF6F54,Handover Date:,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6703ACD4-2DFC-4F23-AFBC-5084D65722E2,NaN,NaN,NaN,text
1187,NaN,NaN,f3245d42-ea77-11e1-aff1-0800200c9a66,Conducted on,NaN,NaN,NaN,True,True,True,...,NaN,NaN,NaN,NaN,NaN,6703ACD4-2DFC-4F23-AFBC-5084D65722E2,NaN,NaN,NaN,datetime


Out of all these attributes, in my opinion, the most meaningful are *item_id*, *label*, *parent_id* and *type*.

In [19]:
df_header[["item_id", "label", "parent_id", "type"]]

,item_id,label,parent_id,type
0,6f6f5e7f-9787-4a06-a4bc-7baf1c5f23b3,Title Page,None,section
1,f3245d46-ea77-11e1-aff1-0800200c9a66,Document No.,6f6f5e7f-9787-4a06-a4bc-7baf1c5f23b3,textsingle
2,f3245d40-ea77-11e1-aff1-0800200c9a66,Audit Title,6f6f5e7f-9787-4a06-a4bc-7baf1c5f23b3,textsingle
3,f3245d41-ea77-11e1-aff1-0800200c9a66,Client / Site,6f6f5e7f-9787-4a06-a4bc-7baf1c5f23b3,textsingle
4,f3245d42-ea77-11e1-aff1-0800200c9a66,Conducted on,6f6f5e7f-9787-4a06-a4bc-7baf1c5f23b3,datetime
...,...,...,...,...
1184,f3245d41-ea77-11e1-aff1-0800200c9a66,Store Name:,6703ACD4-2DFC-4F23-AFBC-5084D65722E2,text
1185,27C5FD11-DD48-4A15-A5AE-C3222C60CFF5,Project Manager:,6703ACD4-2DFC-4F23-AFBC-5084D65722E2,text
1186,A8F9CC14-4BCC-414E-8EA4-12F94EBF6F54,Handover Date:,6703ACD4-2DFC-4F23-AFBC-5084D65722E2,text
1187,f3245d42-ea77-11e1-aff1-0800200c9a66,Conducted on,6703ACD4-2DFC-4F23-AFBC-5084D65722E2,datetime


#### Items

*Items* attribute is similar with *header*.

In [20]:
from pandas.io.json import json_normalize
df_items = pd.concat([json_normalize(template["items"]) for template in data_json], ignore_index = bool)
df_items

/Users/silvia/PycharmProjects/SafetyCulture/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,action_item_profile_id,inactive,item_id,label,options,options.condition,options.drawing_base_image,options.element,options.enable_date,options.enable_signature_timestamp,...,options.visible_in_report,options.weighting,parent_id,reference_item_profile_ids,responses.datetime,responses.name,responses.response,responses.text,responses.value,type
0,NaN,NaN,95594086-34f0-4680-b43b-78fa330c70a0,Audit,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,None,NaN,NaN,NaN,NaN,NaN,NaN,section
1,NaN,NaN,06e129d0-05dc-496b-9c19-ce583fd8ea8e,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,95594086-34f0-4680-b43b-78fa330c70a0,NaN,NaN,NaN,NaN,,NaN,question
2,NaN,NaN,9616bec9-80ed-4589-8182-2ae443dc7bc9,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,95594086-34f0-4680-b43b-78fa330c70a0,NaN,NaN,NaN,NaN,NaN,NaN,category
3,NaN,NaN,2a39efa2-2d5e-4f2b-9342-a8860ea84260,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,9616bec9-80ed-4589-8182-2ae443dc7bc9,NaN,NaN,NaN,NaN,NaN,NaN,address
4,NaN,NaN,7395020e-aa56-48db-a401-ac361ce54c9e,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,9616bec9-80ed-4589-8182-2ae443dc7bc9,NaN,NaN,NaN,NaN,NaN,NaN,scanner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13382,NaN,NaN,77F8D8E3-8F5B-45F8-BB25-88A9731BD667,Ground Anchor Position in housing (Gunnebo pods),NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8952FD67-A221-43A9-96B8-4B5B8FD2236E,NaN,NaN,NaN,NaN,NaN,NaN,media
13383,NaN,NaN,AB416A72-6F5D-48B2-85AF-290BDB1BB5F2,Distance shot showing ATM location within store,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,BD21E1C0-CE9E-4102-A172-2DF34B62204F,NaN,NaN,NaN,NaN,NaN,NaN,question
13384,NaN,NaN,9CD481AF-60B9-4BA5-9230-E3FE8D431554,Distance shot showing ATM location within store,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,BD21E1C0-CE9E-4102-A172-2DF34B62204F,NaN,NaN,NaN,NaN,NaN,NaN,media
13385,NaN,NaN,21A3D435-FB28-46B0-B8D1-5EC830BB2A3F,Comments - please use comments box to note any...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,BD21E1C0-CE9E-4102-A172-2DF34B62204F,NaN,NaN,NaN,NaN,NaN,NaN,text


### Conclusion

Each template is characterized by:
* template id
* name that can be obtained from metadata
* description that can be obtained from metadata
* labels of the items in header and in the template
* item types
* industry and subindustry classification

In [21]:
templates = []
for template in data_json:
    items = []
    for item in template["header"]:
        if ("label" in item) and (len(item["label"])>0) and (len(item["label"])<10000):
            items.append((item["label"], item["type"]))
    for item in template["items"]:
        if ("label" in item) and (len(item["label"])>0) and (len(item["label"])<10000):
            items.append((item["label"], item["type"]))
    
    
    templates.append([template["template_id"], 
                      items, 
                      template["template_data"]["metadata"]["name"],
                      template["template_data"]["metadata"]["description"],
                      template["template_data"]["metadata"]["industry"], 
                      template["template_data"]["metadata"]["subindustry"]])

df_templates = pd.DataFrame(templates, columns = ["id", "items", "name", "description", "industry", "subindustry"])
df_templates.set_index("id", inplace=True)
df_templates

,items,name,description,industry,subindustry
id,,,,,
template_65db0caaac874d08ae7e7d15d05b5c7c,"[(Title Page, section), (Document No., textsin...",All items - duplicate,,-2,-2
template_dcb88e21ea304c9baa842daa2e5abafc,"[(Title Page, section), (Document No., textsin...",Restest - duplicate,,-2,-2
template_58aa02d963444fd9b94c76ea03537d9f,"[(Title Page, section), (Document No., textsin...",bda04557-fed0-4f60-ad3e-ad1d590b79b1,,-2,-2
template_c0ed72130e4f401eb281c5d7e725ec64,"[(Title Page, section), (Conducted on, datetim...",Question and List - duplicate,,-2,-2
template_0D81EB72BFBD4D39ABC14BBB7735691F,"[(Welcome, section), (Welcome!, category), (In...",Advanced Items Template,Beta Testing Template,-2,-2
...,...,...,...,...,...
template_CB3D831B3C25481896BEC6B2C0ED8D57,"[(Rapportgegevens, section), (Naam verzekering...",Rapport van expertise - WB,Versie 24 juni 2015,7,3
template_4fcce8994e9f453c8f87ed7eccc41591,"[(Title Page, section), (Conducted on, datetim...",Grs test 2,,7,3
template_c814e26f64944ee1bde8917a1f3587e2,"[(Information, section), (Document No., textsi...",The new DR!,ewfwef,7,3


# Solution

## We’re interested in seeking solutions that help increase the number of customers using the standard public checklist templates for their inspections (e.g. how can we recommend the suitable templates to customers)?

I suppose that the templates that you have sent are templates created by users, and they are not standard public template. Therefore, in my opinion the user created new templates as they couldn't find a public template suitable because:
* there is no public template suitable for their needs
* they couldn't find the suitable template 


### Create new public templates

To create new public templates that are missing, I suggest to cluster together templates that are similar. To do so, I first represent the templates in a numerical format using the name, descrioption and items labels. 

In [22]:
from nltk.tokenize import word_tokenize
import re 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

from sklearn.feature_extraction.text import TfidfVectorizer


stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer() 

def get_tokens(text):
    return [lemmatizer.lemmatize(word.lower()) for word in word_tokenize(text) if re.match("[a-z]\w{1,}", word.lower()) and (word.lower() not in stop_words)]

def get_corpus():
    corpus = []
    for template_id, template in df_templates.iterrows():
        tokens = []
        tokens.extend(get_tokens(template["name"]))
        tokens.extend(get_tokens(template["description"]))
        for label, _ in template["items"]:
            tokens.extend(get_tokens(label))
        corpus.append(' '.join(tokens))
    return corpus

corpus = get_corpus()

tfidf_vectorizer = TfidfVectorizer(analyzer='word')
corpus_tfidf = tfidf_vectorizer.fit_transform(corpus)
corpus_tfidf.shape

(170, 6151)

Then, I group similar articles together according with their vectorial representation. I use the algorithm Kmeans. I choose this algorithm because is the easy and fast. 

First I need to choose the most suitable number o clusters for the given dataset using the silouette score. 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from tqdm import tqdm 
plt.figure(figsize=(15,5))

silouettes = []
range_silouette=range(10,120)
for n_clusters in tqdm(range_silouette):
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(corpus_tfidf)
    centers = clusterer.cluster_centers_

    score = silhouette_score(corpus_tfidf, preds)
    silouettes.append(score)

plt.plot(range_silouette, silouettes);

 72%|███████▏  | 79/110 [01:44<00:46,  1.50s/it]

Best number of clusters is 101. 

In [ ]:
from sklearn.cluster import KMeans
from collections import Counter

num_clusters = 101

kmeans = KMeans(n_clusters=num_clusters)

kmeans.fit(corpus_tfidf)

clusters = kmeans.labels_.tolist()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(pd.DataFrame(Counter(clusters).items(), columns=["Cluster", "No. items"]).set_index("Cluster").sort_values("No. items", ascending=False))


In [ ]:
df_clusters = pd.DataFrame()
df_clusters['cluster'] = clusters
df_clusters['tokens'] = corpus

In [ ]:
for cluster_nb in range(num_clusters):
    print()
    print("******************************************************************************************")
    print()
    print(f"Cluster: {cluster_nb}")
    for t in df_clusters[df_clusters['cluster'] == cluster_nb]['tokens']:
        print("------------------------------------------------------------------------------------------")
        print(t)

To have a better understanding of what we can find in every cluster, I print the top words according to their Tf-Idf score. 

In [ ]:
from operator import itemgetter

def get_top_words(features, tfidf, ntop):
    return ' '.join([feature for feature, score in sorted(filter(lambda x:x[1]>0, zip(features, tfidf)), key=itemgetter(1), reverse=True)][:ntop])

for cluster_nb in range(num_clusters):
    print()
    print("******************************************************************************************")
    print()
    print(f"Cluster: {cluster_nb}")
    for template_tfidf in corpus_tfidf[(df_clusters['cluster'] == cluster_nb).values]:
        print("------------------------------------------------------------------------------------------")
        print(get_top_words(tfidf_vectorizer.get_feature_names(), template_tfidf.T.todense(), 10))   

Based on the templates in each cluster and the most meaningful words we can observe which templates are missing from our database and have a high demand, and therefore to manually create new templates. 

### Improve the template search

I observed that Safety Culture has a lot of public templates and finding the suitable template can be like searching for a needle in the hay stack.  I suggest to enhance the templates with tags, in this way the search can be made based on the industry and subindustry, as well as on tags. 

The tags can be added to a template manually or automatically, using tf-idf scores or other keyword extraction approches. 

When a customer wants to create a new template, Safety Culture asks him/her to choose the tags that better describe the template that he/she needs, and recommends the top N templates that contain the specified tags. 

## Based on the findings above, do you have any suggestions on potential features/products that can be built to improve our customers’ experiences ?

See previous response.

## Through using the checklists for inspection, can you suggest some potential solutions for correlating the data from the checklists with potential risks relevant to the respective inspections.

I have observed that the responses to a question in a template can be of different types, and each type should be analyzed differently for potential risks. 
* Multiple choice responses - already contaign information about the response that presents risks for the inspection.
* Text answer - we can use Natuarl Language Processing approces to identify risks described in the text.
* Numbers - if the same template is used by many customers, we can collect data related with that question; analysing the data we can detect values that are outliers for the distribution of the values. To do so, for a normal distribution of the values, we compute the mean and the standard error for a question. Those values that are at a distance of more than between (mean - 1.96 * standard error, mean + 1.96 * standard error), for a 95% confidence, are outliers. 
* Checkbox - if the data can be split in a majority with the checkbox selected and only a few customers with the checkbox unselected, or the other way around, we can conclude that there is a potential risk.
* Photos can be used to detect anomalies using image processing.



# Thank you for your time